In [2]:
from functions import (prep, folding, inbu, LSTM_model,setting)
import numpy as np
from matplotlib import pyplot as plt
from keras.models import load_model
from jellyfish import damerau_levenshtein_distance, levenshtein_distance
import pandas as pd
import distance
from sklearn import metrics
np.random.seed(42)
%load_ext autoreload
%autoreload 2

Defining the Setting which log to use and so on

In [4]:
setting_inputs = setting.inputdef()
csvname = f"{setting_inputs['flatten_by']}_complete" if setting_inputs['complete'] else f"{setting_inputs['flatten_by']}_filter"
# Define model_file based on single_log value
model_file = f"{csvname}_single" if setting_inputs['single_log'] else f"{csvname}_enriched"

ValueError: Invalid input: flatten_by must be one of ['Orders', 'Items', 'Packages']

### Loading the data as well as preparing the OCEL and also split the data:

In [5]:
ocel, act_dict, cust_dict = prep.prep_ocel_complete(setting_inputs=setting_inputs,csvname=csvname)
ocel_train, ocel_test = folding.folding_train_test(ocel, csvname= model_file)

Defining values for normalization

In [6]:
divisor = np.mean(ocel['Time_Diff'])  # average time between events
divisor2 = np.mean(ocel['Time_Since_Start'])  # average time between current and first events
divisorTR = np.mean(ocel['Remaining_Time'])  # average time instance remaining
divisor3 = ocel.groupby('Case_ID')['Time_Since_Start'].apply(lambda x: (x.iloc[-1] - x).mean()).mean()

print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
print(f"divisor3: {divisor3}")
print(f'Amount of rows of the OCEL: {len(ocel)}')

divisor: 103436.38622985176
divisor2: 604261.1036767652
divisorTR: 1248093.355995932
divisor3: 1118980.691883163
Amount of rows of the OCEL: 32447


Dimensions for the MatriX as well as the features

In [7]:

num_of_features, max_trace_length, act_feat,cust_feat, target_act_feat, target_act_feat_dict, other_features = setting.feature_dimensios(ocel=ocel,setting_input=setting_inputs)
number_of_train_cases = len(ocel_train)
target_act_length = len(target_act_feat)
print(f"Number of train cases: {number_of_train_cases}, Max trace length: {max_trace_length}, Number of features: {num_of_features}")


Number of train cases: 21620, Max trace length: 42, Number of features: 18


### Generating the Input matrix

In [8]:
X_train,y_train_a, y_train_t, y_train_tr = inbu.generating_inputs(ocel_fold=ocel_train,ocel=ocel,setting_input=setting_inputs,
                                                                  dn= divisor, ds= divisor2, dr= divisorTR)

print(f"Shape of X_train: {X_train.shape}")
print(f"This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): {(number_of_train_cases, max_trace_length, num_of_features)} => {X_train.shape ==(number_of_train_cases, max_trace_length, num_of_features)}")
print(f"Shape of y_train_a: {y_train_a.shape}, this matches the desired shape (number_of_train_cases, target_act_length): {(number_of_train_cases, target_act_length)} => {y_train_a.shape ==(number_of_train_cases, target_act_length)}")
print(f"Shape of y_train_t: {y_train_t.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_t.shape ==(number_of_train_cases, )}")
print(f"Shape of y_train_tr: {y_train_tr.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_tr.shape ==(number_of_train_cases, )}\n")


Shape of X_train: (21620, 42, 18)
This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): (21620, 42, 18) => True
Shape of y_train_a: (21620, 12), this matches the desired shape (number_of_train_cases, target_act_length): (21620, 12) => True
Shape of y_train_t: (21620,), this matches the desired shape (number_of_train_cases, ): (21620,) => True
Shape of y_train_tr: (21620,), this matches the desired shape (number_of_train_cases, ): (21620,) => True



In [12]:
display(ocel_train.loc[0:12,~ocel.columns.str.startswith(('Next_','Cust_','Act_'))].drop(columns=['Items','Packages']))
features_vis = act_feat +['Time_Diff','Time_Since_Start','Time_Since_Midnight','Weekday','Position','In_Package','Amount_Items']

,Case_ID,Activity,Timestamp,Customers,In_Package,Amount_Items,Time_Diff,Time_Since_Start,Time_Since_Midnight,Weekday,Remaining_Time,Position,Trace_Len
0,990001,A,2019-05-20 09:07:47,a,0,4,0,0,32867,0,3197465,1,20
1,990001,C,2019-05-20 11:13:54,a,0,4,7567,7567,40434,0,3189898,2,20
2,990001,B,2019-05-20 11:20:13,a,0,1,379,7946,40813,0,3189519,3,20
3,990001,D,2019-05-20 13:54:37,a,0,1,9264,17210,50077,0,3180255,4,20
4,990001,E,2019-05-21 10:03:49,a,0,1,72552,89762,36229,1,3107703,5,20
5,990001,D,2019-05-23 10:40:50,a,0,1,175021,264783,38450,3,2932682,6,20
6,990001,B,2019-05-23 11:00:25,a,0,1,1175,265958,39625,3,2931507,7,20
7,990001,F,2019-05-23 11:27:55,a,0,4,1650,267608,41275,3,2929857,8,20
8,990001,G,2019-05-23 17:26:26,a,1,5,21511,289119,62786,3,2908346,9,20
9,990001,H,2019-05-23 17:43:12,a,1,5,1006,290125,63792,3,2907340,10,20


In [11]:
pd.DataFrame(X_train[7],  columns=features_vis).rename_axis('Time steps')[-15:-1]

,Act_A,Act_B,Act_C,Act_D,Act_E,Act_F,Act_G,Act_H,Act_I,Act_J,Act_K,Time_Diff,Time_Since_Start,Time_Since_Midnight,Weekday,Position,In_Package,Amount_Items
Time steps,,,,,,,,,,,,,,,,,,
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
34,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.380405,0.000000,1.0,0.0,4.0
35,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073156,0.012523,0.467986,0.000000,2.0,0.0,4.0


### Model creation

In [ ]:
print(f'For the following setting a model is now trained {model_file}\n')
history, best_model_name, early_stopping= LSTM_model.LSTM_MODEL(X_train, y_train_a, y_train_t, y_train_tr,filename=model_file)

### Evaluation

Model history

In [ ]:
val_loss = min(history.history['val_loss'])
val_loss2 = best_model_name.best
epoch = early_stopping.stopped_epoch - 49
print(f'The best value for the validation loss is  {val_loss} and was archived at the epoch {epoch}\n')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

modelname = 'model_' + model_file + f"_{epoch:02d}-{val_loss:.2f}.h5"

print(f'The best model has the name {modelname}\n')

Evaluation

In [ ]:
model = load_model(f'./output_files/models/{modelname}')
X_test, y_test_a, y_test_t, y_test_tr = inbu.generating_inputs(ocel_fold=ocel_test,ocel=ocel,setting_input=setting_inputs,
                                                                  dn= divisor, ds= divisor2, dr= divisorTR)

# y_t = y_t * divisor3
y = model.predict(X_test,verbose=1)
y_char = y[0][:][:]
y_t = y[1][:][:]
y_tr = y[2][:][:]
max_index_list = [np.argmax(pred) for pred in y_char]
pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
y_t = np.maximum(y_t, 0)
y_t1 = y_t * divisor
y_tr1 = y_tr * divisorTR

columns_to_drop = [col for col in ocel_test.columns if 'Act_' in col] + \
                [col for col in ocel_test.columns if 'Cust_' in col] + \
                ['Customers', 'Next_Time_Since_Start',
                'Next_Time_Since_Midnight', 'Next_Weekday',
                'Position', 'Time_Since_Midnight', 'Weekday'] + other_features

columns_to_drop_existing = [col for col in columns_to_drop if col in ocel_test.columns]
output_ocel = ocel_test.drop(columns=columns_to_drop_existing).copy()
output_ocel['Pred_Activity'] = pred_act_list
output_ocel['Pred_Time_Diff'] = y_t1
output_ocel['Pred_Remaining_Time'] = y_tr1

output_ocel['Levenshtein'] = output_ocel.apply(lambda row: 1 - levenshtein_distance(row['Pred_Activity'], row['Next_Activity']), axis=1)
output_ocel['Damerau'] = output_ocel.apply(lambda row: 1 - (damerau_levenshtein_distance(row['Pred_Activity'], row['Next_Activity']) / max(len(row['Pred_Activity']), len(row['Next_Activity']))), axis=1)
output_ocel['Damerau'] = output_ocel['Damerau'].clip(lower=0)
output_ocel['Jaccard'] = output_ocel.apply(lambda row: 1 - distance.jaccard(row['Pred_Activity'], row['Next_Activity']), axis=1)

act_comp = output_ocel['Pred_Activity'] == output_ocel['Next_Activity'] 
print(f'The accuracy of the activation prediction is {sum(act_comp)/len(act_comp)}')
MAE_Time_diff = metrics.mean_absolute_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60),output_ocel['Next_Time_Diff']/ (24 * 60 * 60))
MAE_rem_time = metrics.mean_absolute_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60),output_ocel['Next_Remaining_Time']/ (24 * 60 * 60))
RMSE_Time_diff = metrics.mean_squared_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60),output_ocel['Next_Time_Diff']/ (24 * 60 * 60),squared=False)
RMSE_rem_time = metrics.mean_squared_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60),output_ocel['Next_Remaining_Time']/ (24 * 60 * 60),squared=False)
print(f'MAE of the time between events in days {MAE_Time_diff}')
print(f'MAE of the remaining time in days {MAE_rem_time}')
# print(f'RMSE of the time between events in days {RMSE_Time_diff}')
# print(f'RMSE of the remaining time in days {RMSE_rem_time}')

In [ ]:
modelname = 'model_Items_complete_enriched_24-1.73.h5'

model = load_model(f'./output_files/models/{modelname}')
prefix_length = 2
idx = (ocel_test['Trace_Len'].values >= prefix_length) *(ocel_test['Position'].values == prefix_length)
len(ocel_test.loc[idx,'Case_ID'].unique())

In [ ]:
prefix_lengths = range(2, max_trace_length)  # List of prefix lengths to consider
results = []
print(max_trace_length)
test_val = setting.feature_dimensios(ocel=ocel_test,setting_input=setting_inputs)[1]
for prefix_length in prefix_lengths:    
    if test_val == prefix_length:
        continue
    print(f"Results for Prefix Length {prefix_length}:")
    # Generate inputs with the current prefix length
    X_test, y_test_a, y_test_t, y_test_tr = inbu.generating_inputs_pref(ocel_fold=ocel_test, ocel=ocel,
                                                                  setting_input=setting_inputs,
                                                                  dn=divisor, ds=divisor2, dr=divisorTR,
                                                                  prefix_length=prefix_length)
    # Make predictions with the model
    y = model.predict(X_test, verbose=1)
    y_char, y_t, y_tr = y[0][:][:], y[1][:][:], y[2][:][:]

    max_index_list = [np.argmax(pred) for pred in y_char]
    pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
    y_t = np.maximum(y_t, 0)
    y_t, y_tr = y_t * divisor, y_tr * divisorTR
    y_test_t, y_test_tr = y_test_t * divisor, y_test_tr * divisorTR  
    columns_to_drop = [col for col in ocel_test.columns if 'Act_' in col] + \
                    [col for col in ocel_test.columns if 'Cust_' in col] + \
                    ['Customers', 'Next_Time_Since_Start',
                    'Next_Time_Since_Midnight', 'Next_Weekday',
                    'Position', 'Time_Since_Midnight', 'Weekday'] + other_features
    idx = (ocel_test['Trace_Len'].values >= prefix_length) * (ocel_test['Position'].values == prefix_length)
    next_act = ocel_test.loc[idx,'Next_Activity'] 
    act_comp = pred_act_list == next_act
    mae_time_diff = metrics.mean_absolute_error(y_t / (24 * 60 * 60), y_test_t / (24 * 60 * 60))
    mae_remaining_time = metrics.mean_absolute_error(y_tr / (24 * 60 * 60), y_test_tr / (24 * 60 * 60))
    rmse_time_diff = metrics.mean_squared_error(y_t / (24 * 60 * 60), y_test_t / (24 * 60 * 60), squared=False)
    rmse_remaining_time = metrics.mean_squared_error(y_tr / (24 * 60 * 60), y_test_tr / (24 * 60 * 60), squared=False)
    # Store the results for the current prefix length
    results.append({
        'Prefix Length': prefix_length,
        'length': sum(idx),
        'Accuracy Activity': sum(act_comp)/len(act_comp),
        'MAE Time Difference': mae_time_diff,
        'MAE Remaining Time': mae_remaining_time,
        'RMSE Time Difference': rmse_time_diff,
        'RMSE Remaining Time': rmse_remaining_time
    })

# Output the overall results
df_results = pd.DataFrame(results)
print("\nOverall Results:")
display(df_results)
df_results.mean()

In [ ]:
prefix_lengths = range(2, max_trace_length)  # List of prefix lengths to consider

MAE_Rem = []
act_acc = []
MAE_next = []
test_val = setting.feature_dimensios(ocel=ocel_test,setting_input=setting_inputs)[1]
print(max_trace_length)
for prefix_length in prefix_lengths:    
    if test_val == prefix_length:
        continue
    print(f"Results for Prefix Length {prefix_length}:")
    # Generate inputs with the current prefix length
    X_test, y_test_a, y_test_t, y_test_tr = inbu.generating_inputs_pref(ocel_fold=ocel_test, ocel=ocel,
                                                                  setting_input=setting_inputs,
                                                                  dn=divisor, ds=divisor2, dr=divisorTR,
                                                                  prefix_length=prefix_length)
    # Make predictions with the model
    y = model.predict(X_test, verbose=1)
    y_char, y_t, y_tr = y[0][:][:], y[1][:][:], y[2][:][:]

    max_index_list = [np.argmax(pred) for pred in y_char]
    pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
    y_t = np.maximum(y_t, 0)
    y_t, y_tr = y_t * divisor, y_tr * divisorTR
    y_test_t, y_test_tr = y_test_t * divisor, y_test_tr * divisorTR  

    idx = (ocel_test['Trace_Len'].values >= prefix_length) * (ocel_test['Position'].values == prefix_length)

    next_act = ocel_test.loc[idx,'Next_Activity'] 
    act_comp = pred_act_list == next_act
    mae_time_diff = [np.abs(x/ (24 * 60 * 60) -  y/ (24 * 60 * 60)) for x,y in zip(y_t,y_test_t)]
    mae_remaining_time = [np.abs(x / (24 * 60 * 60) - y / (24 * 60 * 60)) for x,y in zip(y_tr,y_test_tr)]
    # rmse_time_diff = [np.abs(x / (24 * 60 * 60)-  y / (24 * 60 * 60), squared=False) for x,y in zip(y_t,y_test_t)]
    # rmse_remaining_time = [np.abs(x / (24 * 60 * 60) -  y / (24 * 60 * 60), squared=False) for x,y in zip(y_t,y_test_t)]
    # Store the results for the current prefix length
    act_acc.extend(act_comp)
    MAE_next.extend(mae_time_diff)
    MAE_Rem.extend(mae_remaining_time)


print(sum(act_acc)/len(act_acc))
print(sum(MAE_next)/len(act_acc))
print(sum(MAE_Rem)/len(act_acc))


In [ ]:
print(sum(act_acc)/len(act_acc))
print(sum(MAE_next)/len(MAE_next))
print(sum(MAE_Rem)/len(MAE_Rem))